In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import datetime as dt 
import seaborn as sns
import osmnx as ox
from collections import defaultdict
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity
import folium
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statistics import mean
import matplotlib.dates as mdates


In [2]:
df = pd.read_csv('/Users/mariu/Desktop/Masterarbeit/archive/data_start.csv')

In [3]:

# 1. Datenvorbereitung: Auswahl der relevanten Spalten
relevant_columns = ['start_station_id', 'cluster']

# 2. Häufigkeitsanalyse
# Zählen Sie, wie oft jede Station in jedem Cluster als Start- oder Endstation auftritt
station_counts = df[relevant_columns].melt(id_vars=['cluster'], value_name='station').groupby(['cluster', 'station']).size().reset_index(name='count')

# Sortieren der Daten für Pareto-Analyse
station_counts = station_counts.sort_values(['cluster', 'count'], ascending=[True, False])

# 3. Pareto-Analyse: Ermittlung der kumulativen Prozentzahl und Auswahl der wichtigsten Stationen pro Cluster
station_counts['cumulative_percentage'] = station_counts.groupby('cluster')['count'].apply(lambda x: x.cumsum() / x.sum() * 100)

# Identifizierung der wichtigsten Stationen (z.B. top 20%)
threshold = 40
important_stations = station_counts[station_counts.groupby('cluster')['cumulative_percentage'].apply(lambda x: x <= threshold)]

print(important_stations)

     cluster       station  count  cumulative_percentage
98         0        WL-012  21208               3.064408
4          0         13016  20311               5.999205
1          0         13001  19673               8.841816
24         0         13430  19155              11.609580
3          0         13011  18879              14.337463
..       ...           ...    ...                    ...
492        3         13379   9595              34.150247
607        3  TA1308000029   9431              35.507118
600        3  TA1307000124   9096              36.815792
472        3         13196   9053              38.118278
463        3         13132   7915              39.257037

[79 rows x 4 columns]


/var/folders/y8/j3l3spt14jd5q6r19v3l8_800000gn/T/ipykernel_2870/2573352830.py:12: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  station_counts['cumulative_percentage'] = station_counts.groupby('cluster')['count'].apply(lambda x: x.cumsum() / x.sum() * 100)
/var/folders/y8/j3l3spt14jd5q6r19v3l8_800000gn/T/ipykernel_2870/2573352830.py:16: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt t